<a href="https://colab.research.google.com/github/HanzlaJavaid/Algorithms/blob/master/ANN_META.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
u0_df = pd.read_csv("/content/drive/MyDrive/Dataset/user_a.csv")
u0_df["user"] = 0
u1_df = pd.read_csv("/content/drive/MyDrive/Dataset/user_b.csv")
u1_df["user"] = 1
u2_df = pd.read_csv("/content/drive/MyDrive/Dataset/user_c.csv")
u2_df["user"] = 2
u3_df = pd.read_csv("/content/drive/MyDrive/Dataset/user_d.csv")
u3_df["user"] = 3

In [ ]:
collective_df = pd.concat([u0_df,u1_df,u2_df,u3_df])

In [ ]:
collective_df.head()

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,F7 theta std,F7 theta m,F7 alpha std,F7 alpha m,F7 beta std,F7 beta m,F3 delta std,F3 delta m,F3 theta std,F3 theta m,F3 alpha std,F3 alpha m,F3 beta std,F3 beta m,FC5 delta std,FC5 delta m,FC5 theta std,FC5 theta m,FC5 alpha std,FC5 alpha m,FC5 beta std,FC5 beta m,T7 delta std,T7 delta m,T7 theta std,T7 theta m,T7 alpha std,T7 alpha m,T7 beta std,...,T8 delta m,T8 theta std,T8 theta m,T8 alpha std,T8 alpha m,T8 beta std,T8 beta m,FC6 delta std,FC6 delta m,FC6 theta std,FC6 theta m,FC6 alpha std,FC6 alpha m,FC6 beta std,FC6 beta m,F4 delta std,F4 delta m,F4 theta std,F4 theta m,F4 alpha std,F4 alpha m,F4 beta std,F4 beta m,F8 delta std,F8 delta_m,F8 theta std,F8 theta m,F8 alpha std,F8 alpha m,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,user
0,1.0,3569.164550,2063.892754,1.673726,4.444736,0.526209,3.002088,1.425022,3.302739,3563.803888,2060.239057,1.404089,2.279398,1.297401,1.960079,0.759956,2.038977,3709.009712,2180.647682,19.491141,39.847419,20.493348,41.228796,46.007372,73.493233,3567.365265,2062.483097,1.209996,1.771811,0.161793,1.818490,0.688672,1.641767,3559.219279,2057.363361,1.027551,2.502389,1.495156,2.207701,1.029114,...,2138.663493,19.310670,36.200836,15.881327,38.774853,43.601995,69.778363,3694.245877,2174.777021,18.426524,40.488393,16.410505,38.850336,45.329263,72.907800,3577.521434,2067.996962,1.232973,3.073659,0.919419,2.634124,3.348698,5.380652,3665.336755,2157.675046,15.543111,37.374199,15.907312,40.854795,45.468326,72.508750,3701.186330,2182.676835,18.192418,41.349662,16.004756,42.046467,46.280843,73.565719,0
1,1.0,3568.423670,2063.099248,1.897790,3.728823,1.304186,1.854353,1.366575,2.546458,3563.560922,2059.969372,0.694171,1.971073,0.756398,1.637064,0.963158,2.117719,3721.781859,2177.507069,7.343344,39.152753,20.145638,32.934098,38.525474,68.196685,3566.746758,2061.497984,0.740290,1.815878,1.153479,1.583097,0.574245,1.616353,3559.526890,2057.324677,0.986897,2.406373,0.779043,2.993743,1.090381,...,2136.305537,9.666578,34.522405,17.767788,26.998472,36.178953,64.661308,3708.391961,2171.961977,10.150358,39.040007,16.553371,28.600763,38.483312,67.962618,3578.776662,2068.731565,1.218218,2.128704,0.352964,3.215416,3.412938,4.803989,3679.285737,2156.011819,9.227783,35.553815,18.089263,28.408800,36.551948,66.931186,3725.210509,2180.197439,8.820788,38.012788,19.601233,29.431054,38.559351,67.470041,0
2,1.0,3568.157929,2062.445859,2.798014,2.574504,1.120537,1.958819,0.982433,2.258622,3563.279981,2059.543651,0.493677,1.222808,0.955544,2.642443,0.725119,1.846020,3723.289253,2176.284578,18.590902,25.349995,17.772827,19.884104,44.163109,68.365284,3566.653610,2061.533231,0.593558,1.830077,0.839921,1.533928,1.032268,1.885158,3558.225822,2057.441208,0.899122,2.312432,1.325438,2.295383,0.908968,...,2135.555927,18.112467,23.514646,9.089433,20.463689,40.165128,64.768686,3708.552141,2169.886880,19.450478,24.764949,15.423059,20.797828,42.884410,68.191273,3580.053926,2070.314385,1.129198,2.083349,1.628629,2.715756,3.462284,4.599409,3679.393107,2153.555265,19.979258,24.464488,13.488471,17.791111,40.754308,66.816547,3724.417296,2176.823208,18.159202,23.612639,14.378291,19.555084,43.210004,67.781924,0
3,1.0,3567.710021,2062.112673,2.181775,3.610507,0.629608,2.155876,0.856275,2.233711,3562.787801,2059.317489,0.509250,1.478831,0.848642,2.539217,1.013335,1.562917,3727.061652,2181.935715,16.831394,32.474502,17.477847,23.278668,41.635879,65.650281,3566.313364,2061.497351,1.220105,1.708304,0.681477,1.139439,1.022393,1.847746,3557.952287,2057.698035,0.944458,2.068626,0.972430,1.510003,1.146271,...,2138.967337,19.870080,28.164362,13.937815,19.992277,37.654372,61.869855,3712.709484,2174.256181,19.891011,31.719748,15.828046,26.779568,40.708965,64.960239,3580.174518,2070.019706,0.933352,1.358187,1.487142,2.762906,2.735184,4.474138,3685.0

In [ ]:
collective_df["actual_class"] = collective_df["Class"].astype(str) + "_" + collective_df["user"].astype(str)

In [ ]:
collective_df.drop(["Class"],inplace=True,axis = 1)

In [ ]:
genarlization_df =collective_df.loc[collective_df["user"] == 1]

In [ ]:
train_df = collective_df.loc[collective_df["user"] != 1]
#ds_train,ds_test,_,_ = train_test_split(train_df.values[:,1:],train_df.values[:,-1],test_size=0.2)

In [ ]:
train_df.drop(["user"],axis=1,inplace=True)
genarlization_df.drop(["user"],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
genarlization_arr = genarlization_df.values[:,:]

In [ ]:
ds_train,ds_test,_,_ = train_test_split(train_df.values[:,:],train_df.values[:,-1],test_size=0.1)

In [ ]:
ds_train.shape

(7776, 113)

In [ ]:
learning_rate = 0.001
meta_step_size = 0.10

inner_batch_size = 25
eval_batch_size = 25

meta_iters = 3000
eval_iters = 10
inner_iters = 10

eval_interval = 250
train_shots = 30
shots = 30
classes = 3

In [ ]:
class Dataset:
  def __init__(self,training,gen_test=False):
    if training:
      ds = ds_train
    else:
      ds = ds_test

    if gen_test:
      ds = genarlization_arr

    x = ds[:,:-1]
    y = ds[:,-1]

    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)

    dataset = tf.data.Dataset.from_tensor_slices((x,y))

    self.data = {}

    for img,label in dataset:
      image = img.numpy()
      lab = str(label.numpy().decode("utf-8"))
      if lab not in self.data:
        self.data[lab] = []
      self.data[lab].append(image)
    self.labels = list(self.data.keys())

  def get_mini_dataset(self, batch_size, repetitions, shots, num_classes, split=False):
        temp_labels = np.zeros(shape=(num_classes * shots))
        temp_images = np.zeros(shape=(num_classes * shots, 112))
        if split:
            test_labels = np.zeros(shape=(num_classes))
            test_images = np.zeros(shape=(num_classes, 112))

        label_subset = random.choices(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
          
            temp_labels[class_idx * shots : (class_idx + 1) * shots] = class_idx
            
            if split:
                test_labels[class_idx] = class_idx
                images_to_split = random.choices(
                    self.data[label_subset[class_idx]], k=shots + 1
                )
                test_images[class_idx] = images_to_split[-1]
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = images_to_split[:-1]
            else:
               
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = random.choices(self.data[label_subset[class_idx]], k=shots)

        dataset = tf.data.Dataset.from_tensor_slices(
            (temp_images.astype(np.float32), temp_labels.astype(np.int32))
        )
        dataset = dataset.shuffle(100).batch(batch_size).repeat(repetitions)
        if split:
            return dataset, test_images, test_labels
        return dataset

In [ ]:
train_dataset = Dataset(training=True)
test_dataset = Dataset(training=False)

In [ ]:
def ann_bn(x,n):
    x = layers.Dense(n)(x)
    x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

def dropout_layer(x,n):
    return layers.Dropout(0.3)

inputs = layers.Input(shape=(112,))
x = ann_bn(inputs,512)
x = ann_bn(x,768)
x = ann_bn(x,1024)
x = ann_bn(x,512)
x = ann_bn(x,64)
outputs = layers.Dense(classes, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

In [ ]:
training = []
testing = []
for meta_iter in range(meta_iters):
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    # Temporarily save the weights from the model.
    old_vars = model.get_weights()
    # Get a sample from the full dataset.
    mini_dataset = train_dataset.get_mini_dataset(
        inner_batch_size, inner_iters, train_shots, classes
    )
    for images, labels in mini_dataset:
        with tf.GradientTape() as tape:
            preds = model(images)
            loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    new_vars = model.get_weights()
    # Perform SGD for the meta step.
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + (
            (new_vars[var] - old_vars[var]) * cur_meta_step_size
        )
    # After the meta-learning step, reload the newly-trained weights into the model.
    model.set_weights(new_vars)
    # Evaluation loop
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_dataset, test_dataset):
            # Sample a mini dataset from the full dataset.
            train_set, test_images, test_labels = dataset.get_mini_dataset(
                eval_batch_size, eval_iters, shots, classes, split=True
            )
            old_vars = model.get_weights()
            # Train on the samples and get the resulting accuracies.
            for images, labels in train_set:
                with tf.GradientTape() as tape:
                    preds = model(images)
                    loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
            test_preds = model.predict(test_images)
            test_preds = tf.argmax(test_preds).numpy()
            num_correct = (test_preds == test_labels).sum()
            # Reset the weights after getting the evaluation accuracies.
            model.set_weights(old_vars)
            accuracies.append(num_correct / classes)
        training.append(accuracies[0])
        testing.append(accuracies[1])
        if meta_iter % 100 == 0:
            print(
                "batch %d: train=%f test=%f" % (meta_iter, accuracies[0], accuracies[1])
            )

batch 0: train=0.666667 test=1.000000
batch 500: train=1.000000 test=1.000000
batch 1000: train=0.333333 test=0.333333
batch 1500: train=1.000000 test=0.333333
batch 2000: train=1.000000 test=0.333333
batch 2500: train=1.000000 test=0.333333


In [ ]:
from sklearn.metrics import accuracy_score
def test_generalization(n):
    gen_accuracy = []
    gen_dataset = Dataset(training=False,gen_test=True) 
    for i in range (0,n):
      train_set, test_images, test_labels = gen_dataset.get_mini_dataset(eval_batch_size, eval_iters, shots, classes, split=True)
      for images, labels in train_set:
        with tf.GradientTape() as tape:
            preds = model(images)
            loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
      test_preds = model.predict(test_images)
      test_preds = tf.argmax(test_preds).numpy()
      print(test_preds)
      print(test_labels)
      acc = accuracy_score(test_preds,test_labels)
      print(acc)
      gen_accuracy.append(acc)
    return sum(gen_accuracy)/len(gen_accuracy)

In [ ]:
gen_accuracy = test_generalization(10)

[1 2 2]
[0. 1. 2.]
0.3333333333333333
[0 1 2]
[0. 1. 2.]
1.0
[0 0 2]
[0. 1. 2.]
0.6666666666666666
[0 1 0]
[0. 1. 2.]
0.6666666666666666
[0 1 2]
[0. 1. 2.]
1.0
[0 2 0]
[0. 1. 2.]
0.3333333333333333
[0 1 2]
[0. 1. 2.]
1.0
[0 1 2]
[0. 1. 2.]
1.0
[0 1 2]
[0. 1. 2.]
1.0
[0 1 2]
[0. 1. 2.]
1.0


In [ ]:
gen_accuracy

0.8